# isce2

## prepare data

In [ ]:
import os
os.chdir('/media/ly/文件/isce_cat')

In [ ]:
# os.chdir('/media/ly/文件/xsc/isce_stack/DEM/')
# !fixImageXml.py -i demLat_N30_N33_Lon_E101_E105.dem.wgs84 -f
# os.chdir('/media/ly/文件/xsc/isce_stack/')

In [ ]:
# !stackSentinel.py -h

In [ ]:
# !stackSentinel.py -s slc -d DEM/demLat_N30_N33_Lon_E101_E105.dem.wgs84 -o aux/aux_poeorb/ -a aux/aux_cal/ -b '31.569715 31.577582 103.412374 103.419046' -W interferogram -C geometry -r 15 -z 5 -f 0.5 -c 4
# !stackSentinel.py -s slc -d DEM/demLat_N30_N33_Lon_E101_E105.dem.wgs84 -o aux/aux_poeorb/ -a aux/aux_cal/ -b '31.569715 31.577582 103.412374 103.419046' -W interferogram -C geometry -r 7 -z 2 -f 0.5 -c 4
!stackSentinel.py -s zip -d dem/demLat_N27_N32_Lon_E100_E106.dem.wgs84 -o orbits/ -a aux_cal/ -b '30.780067 31.130474 102.589458 102.850073' -W interferogram -C geometry -r 20 -z 5 -f 0.5 -c 1

## run stack

In [ ]:
os.chdir('/media/ly/文件/isce_cat/run_files/')

In [ ]:
!./run_1_unpack_slc_topo_master

In [ ]:
!./run_2_average_baseline

In [ ]:
!./run_3_geo2rdr_resample

In [ ]:
!./run_4_extract_stack_valid_region

In [ ]:
!./run_5_merge_master_slave_slc

In [ ]:
!./run_6_merge_burst_igram

In [ ]:
!./run_7_filter_coherence

In [ ]:
!./run_8_unwrap

## prep files for MintPy

In [ ]:
# directory need to copy
# baselines  master  merged (geom_master interferograms)

import os
import glob
import shutil

def copy_files(stack_path, save_path):
    # copy baselines
    baselines_src = os.path.join(stack_path, 'baselines')
    baselines_dst = os.path.join(save_path, 'baselines')
    if os.path.isdir(baselines_src):
        shutil.copytree(baselines_src, baselines_dst)
    
    # copy master
    master_src = os.path.join(stack_path, 'master')
    master_dst = os.path.join(save_path, 'master')
    if os.path.isdir(master_src):
        shutil.copytree(master_src, master_dst)
    
    # copy merged/geom_master
    merged_dst = os.path.join(save_path, 'merged')
    os.mkdir(merged_dst)
    geom_master_src = os.path.join(stack_path, 'merged', 'geom_master')
    geom_master_dst = os.path.join(merged_dst, 'geom_master')
    if os.path.isdir(geom_master_src):
        shutil.copytree(geom_master_src, geom_master_dst)
    
    # copy merged/interferograms
    files = [
        'filt_fine.cor', 'filt_fine.cor.vrt', 'filt_fine.cor.xml',
        'filt_fine.unw', 'filt_fine.unw.vrt', 'filt_fine.unw.xml',
        'filt_fine.unw.conncomp', 'filt_fine.unw.conncomp.vrt', 'filt_fine.unw.conncomp.xml'
    ]
    interferograms_dst = os.path.join(merged_dst, 'interferograms')
    os.mkdir(interferograms_dst)
    interferograms_src = os.path.join(stack_path, 'merged', 'interferograms')
    ifgs = os.listdir(interferograms_src)
    if ifgs:
        for ifg in ifgs:
            ifg_src = os.path.join(interferograms_src, ifg)
            ifg_dst = os.path.join(interferograms_dst, ifg)
            os.mkdir(ifg_dst)
            for file in files:
                file_src = os.path.join(ifg_src, file)
                file_dst = os.path.join(ifg_dst, file)
                shutil.copy(file_src, file_dst)

In [ ]:
stack_path = '/media/ly/文件/res2'
save_path = '/media/ly/file/sc1/isce_stack'
copy_files(stack_path, save_path)

## quickview

In [ ]:
# %matplotlib inline
import matplotlib
matplotlib.use('Agg') 
from osgeo import gdal
import matplotlib.pyplot as plt
import numpy as np
import glob


def view_unw(unws, out_dir):
    if unws:
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        for unw in unws:
            ds = gdal.Open(unw, gdal.GA_ReadOnly)
            data = ds.GetRasterBand(2).ReadAsArray()
            ds = None
            fig,ax = plt.subplots(figsize=(11,10))
            m = np.max(np.abs(data))
            im = ax.imshow(data, cmap='jet', vmin=-m, vmax=m)
            fig.colorbar(im)
            ax.set_title('unwraped')
            unw_name = unw.split('/')[-2] + '_unw'
            fig.savefig(os.path.join(out_dir, unw_name))
    else:
        print('no unw found.')

    
def view_ifg(ifgs, out_dir):
    if ifgs:
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        for ifg in ifgs:
            ds = gdal.Open(ifg, gdal.GA_ReadOnly)
            data = ds.GetRasterBand(1).ReadAsArray()
            ds = None
            fig,ax = plt.subplots(figsize=(11,10))
            im = ax.imshow(np.angle(data), cmap='jet')
            fig.colorbar(im)
            ax.set_title('interferogram')
            ifg_name = ifg.split('/')[-2] + '_ifg'
            fig.savefig(os.path.join(out_dir, ifg_name))
    else:
        print('no ifgs found.')

    
def view_cor(cors, out_dir):
    if cors:
        if not os.path.isdir(out_dir):
            os.mkdir(out_dir)
        for cor in cors:
            ds = gdal.Open(cor, gdal.GA_ReadOnly)
            data = ds.GetRasterBand(1).ReadAsArray()
            ds = None
            fig,ax = plt.subplots(figsize=(11,10))
            im = ax.imshow((data), cmap='gray')
            fig.colorbar(im)
            ax.set_title('coherence')
            fig.savefig(out_file)
            cor_name = ifg.split('/')[-2] + '_cor'
            fig.savefig(os.path.join(out_dir, cor_name))
    else:
        print('no cors found.')


In [ ]:
interferograms_dir = '/media/ly/文件/MintPy/xsc/isce_stack/stack_res_1/merged/interferograms'
unws = glob.glob(interferograms_dir + '/*/filt_fine.unw.vrt')
ifgs = glob.glob(interferograms_dir + '/*/filt_fine.int.vrt')
cors = glob.glob(interferograms_dir + '/*/filt_fine.cor.vrt')

out_dir = '/media/ly/文件/MintPy/xsc/isce_stack/stack_res_1/merged/quickview'

view_unw(unws, out_dir)
view_ifg(ifgs, out_dir)
view_cor(cors, out_dir)

## remove ifgs

In [ ]:
import os
import shutil


def remove_ifg(origin_ifgs_path, removed_ifgs_path, ifgs):
    if not os.path.isdir(removed_ifgs_path):
        os.mkdir(removed_ifgs_path)
    for ifg in ifgs:
        ifg_path = os.path.join(origin_ifgs_path, ifg)
        if os.path.isdir(ifg_path):
            shutil.move(ifg_path, removed_ifgs_path)
        else:
            print(ifg + ' not exists.')

ifgs = [
    '20170120_20170303','20170219_20170327','20170315_20170408','20170923_20171005','20170923_20171017',
    '20170923_20171110','20171204_20180121','20180614_20180626','20181105_20181223','20190116_20190128',
    '20190221_20190422','20190808_20190925','20190820_20190925','20190901_20190925','20190913_20190925',
    '20190913_20191019','20190913_20191031','20191007_20191124','20191019_20191206','20191112_20191206',
    '20191112_20200111','20191124_20200111','20200228_20200404','20200228_20200416','20200522_20200615',
    '20200603_20200627'
]

origin_ifgs_path = '/media/ly/file/sc1/isce_stack/merged/interferograms'
removed_ifgs_path = '/media/ly/file/sc1/isce_stack/merged/removed_ifgs'

remove_ifg(origin_ifgs_path, removed_ifgs_path, ifgs)

# MintPy

## run sbas

In [ ]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from mintpy.objects import ifgramStack
from mintpy.utils import plot as pp, utils as ut
from mintpy import view, plot_network

prj_dir = '/media/ly/文件/sc2/mintpy6'
if not os.path.isdir(prj_dir):
    os.mkdir(prj_dir)

os.chdir(prj_dir)                        # work directory
prj_file = 'sc26.txt'                    # configure file

In [ ]:
config = """##-------------------------------- MintPy -----------------------------##
########## 1. Load Data (--load to exit after this step)
## load_data.py -H to check more details and example inputs.
mintpy.load.processor        = isce
##---------for ISCE only:
mintpy.load.metaFile         = ../isce_stack/master/IW*.xml
mintpy.load.baselineDir      = ../isce_stack/baselines
##---------interferogram datasets:
mintpy.load.unwFile          = ../isce_stack/merged/interferograms/*/filt_*.unw
mintpy.load.corFile          = ../isce_stack/merged/interferograms/*/filt_*.cor
mintpy.load.connCompFile     = ../isce_stack/merged/interferograms/*/filt_*.unw.conncomp
##---------geometry datasets:
mintpy.load.demFile          = ../isce_stack/merged/geom_master/hgt.rdr
mintpy.load.lookupYFile      = ../isce_stack/merged/geom_master/lat.rdr
mintpy.load.lookupXFile      = ../isce_stack/merged/geom_master/lon.rdr
mintpy.load.incAngleFile     = ../isce_stack/merged/geom_master/los.rdr
mintpy.load.azAngleFile      = ../isce_stack/merged/geom_master/los.rdr
mintpy.load.shadowMaskFile   = ../isce_stack/merged/geom_master/shadowMask.rdr
mintpy.load.waterMaskFile    = None
##---------subset (optional):
mintpy.subset.lalo 			= 30.522521:30.915466,102.775720:103.124288

########## 2. modify_network
mintpy.network.keepMinSpanTree 		= yes  #[yes / no], auto for yes, keep interferograms in Min Span Tree network

########## 3. reference_point

########## 4. correct_unwrap_error (optional)

########## 5. invert_network
mintpy.networkInversion.minTempCoh  	= 0.5 #[0.0-1.0], auto for 0.7, min temporal coherence for mask

########## 6. correct_troposphere (optional and recommended)
mintpy.troposphericDelay.method         = height_correlation   #[pyaps / height_correlation / no], auto for pyaps

########## 7. deramp (optional)
mintpy.deramp                           = linear

########## 8. correct_topography (optional and recommended)

########## 9.1 residual_RMS (root mean squares for noise evaluation)

########## 9.2 reference_date

########## 10. velocity

########## 11.1 geocode (post-processing)

########## 11.2 google_earth (post-processing)

########## 11.3 hdfeos5 (post-processing)

########## 11.4 plot
"""
# with open(prj_file, 'w+') as f:
#     f.write(config)

### load_data

In [ ]:
!smallbaselineApp.py $prj_file --dostep load_data

In [ ]:
view.main('./inputs/ifgramStack.h5 20170120_20170213 --nrows 1 --ncols 3'.split())

In [ ]:
!info.py ./inputs/geometryRadar.h5 --compact

In [ ]:
# view.main('./inputs/geometryRadar.h5 --notick --noaxis'.split())

### modify_network

In [ ]:
!smallbaselineApp.py $prj_file --dostep modify_network

In [ ]:
plot_network.main(['./inputs/ifgramStack.h5'])

### reference_point

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_point

In [ ]:
!info.py ./inputs/ifgramStack.h5 | egrep 'REF_'    # attribute related to the reference point

### correct_unwrap_error

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_unwrap_error

### invert_network

In [ ]:
!smallbaselineApp.py $prj_file --dostep invert_network

In [ ]:
view.main('timeseries.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

In [ ]:
view.main('temporalCoherence.h5 --notick --noaxis --noverbose'.split())
view.main('maskTempCoh.h5 --notick --noaxis --noverbose'.split())

### correct_troposphere

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_troposphere

In [ ]:
view.main('timeseries_ERA5.h5 --wrap --wrap-range -5 5 -u cm --notitle --notick --noaxis'.split())

### deramp

In [ ]:
!smallbaselineApp.py $prj_file --dostep deramp

### correct_topography

In [ ]:
!smallbaselineApp.py $prj_file --dostep correct_topography

In [ ]:
view.main('timeseries_ERA5_ramp_demErr.h5 --notitle --wrap --wrap-range -5 5 --notick --noaxis'.split())

### residual_RMS

In [ ]:
!smallbaselineApp.py $prj_file --dostep residual_RMS

In [ ]:
!head rms_timeseriesResidual_ramp.txt

### reference_date

In [ ]:
!cat reference_date.txt    #auto selected optimal reference date.

In [ ]:
## plot the RMS value
from mintpy.timeseries_rms import plot_rms_bar
txtContent = np.loadtxt('./rms_timeseriesResidual_ramp.txt', dtype=bytes).astype(str)
rms_list = [float(i) for i in txtContent[:, 1]]
date_list = [i for i in txtContent[:, 0]]
fig, ax = plt.subplots(figsize=[10, 4])
ax = plot_rms_bar(ax, date_list, rms_list)
plt.show()

In [ ]:
!smallbaselineApp.py $prj_file --dostep reference_date

### velocity

In [ ]:
!smallbaselineApp.py $prj_file --dostep velocity

In [ ]:
!info.py velocity.h5 --compact

In [ ]:
view.main('velocityERA5.h5 --notick --noaxis --noverbose'.split())

### geocode

In [ ]:
!smallbaselineApp.py $prj_file --dostep geocode

In [ ]:
view.main('./geo/geo_velocity.h5 velocity --dem ./geo/geo_geometryRadar.h5 --figsize 12 12 --notick --noaxis'.split())

### google_earth

In [ ]:
!smallbaselineApp.py $prj_file --dostep google_earth

### save kmz

In [ ]:
# !save_kmz.py -h

In [ ]:
!save_kmz.py ./geo/geo_velocity.h5 -u mm  --noreference -v -60 60 -c jet -o ./geo/geo_velocity_mm

In [ ]:
# !save_kmz_timeseries.py -h

In [ ]:
!save_kmz_timeseries.py ./geo/geo_timeseries_ERA5_ramp_demErr.h5 --steps 20 5 1 --cutoff 0

### plot_smallbaselineApp

In [ ]:
# !plot_smallbaselineApp.sh

## post-processing

### time-series displacement

In [ ]:
%matplotlib inline
import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
from mintpy.utils import utils as ut, plot as pp
from mintpy.defaults.plot import *


def datetime2str(datetime_list):
    date_str = []
    for d in datetime_list:
        date_str.append(d.strftime('%Y%m%d'))
    return date_str


def compare_displacement(ys, xs, ts_files, figsize=(15,7), y_lim=[-100, 100]):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title('time series displacement', fontsize=40)
    ax.set_xlabel('date', fontsize=30)
    ax.set_ylabel('displacrment (mm)', fontsize=30)
    
    ax.set_ylim(y_lim[0], y_lim[1])
    ax.set_aspect('0.2')
    ax.minorticks_on()
    ax.xaxis.grid(True, which='both')
    ax.xaxis.set_tick_params(rotation=30, labelsize=15)
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=15)
    ax.set_xmargin(0.02)
    
    for ts_file in ts_files:
        for y, x in zip(ys, xs):
            date, dis = ut.read_timeseries_yx(y=y, x=x, ts_file=ts_file)
            date_str = datetime2str(date)
            disp_type = ts_file.split('_')[-3]
            label = disp_type + f" Point({y},{x})"
            ax.plot(date_str, dis * 1000, label=label, marker='o')
            ax.xaxis.set_ticks(date_str[::4])
    ax.legend(loc='best', fontsize=20, ncol=2)
    fig.show()
    
    
def plot_displacement(ys, xs, tsfile, figsize=(15,7), y_lim=[-100, 100]):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title('time series displacement', fontsize=40)
    ax.set_xlabel('date', fontsize=30)
    ax.set_ylabel('displacrment (mm)', fontsize=30)

    ax.set_ylim(y_lim[0], y_lim[1])
    ax.set_aspect('0.2')
    ax.minorticks_on()
    ax.xaxis.grid(True, which='both')
    ax.xaxis.set_tick_params(rotation=30, labelsize=15)
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=15)
    ax.set_xmargin(0.02)

    for y, x in zip(ys, xs):
        date, dis = ut.read_timeseries_yx(y=y, x=x, ts_file=tsfile)
        date_str = datetime2str(date)
        label = "Point ({}, {})".format(y, x)
        ax.plot(date_str, dis * 1000, label=label, marker='o')
        ax.xaxis.set_ticks(date_str[::4])
    ax.legend(loc='best', fontsize=20, ncol=2)
    fig.show()
#     fig.savefig('disp.png', dpi=600)
    

In [ ]:
proj_dir = '/media/ly/文件/sc2/mintpy6/'
    
ts_files = [
    os.path.join(proj_dir, 'geo0.5/geo_timeseries_tropHgt_ramp_demErr.h5'),
    os.path.join(proj_dir, 'geo/geo_timeseries_GACOS_ramp_demErr.h5')
]
ys =[520]
xs = [1100]

# compare_displacement(ys, xs, ts_files, figsize=(25, 15), y_lim=[-200, 100])

plot_displacement(ys, xs, ts_files[0], figsize=(25, 15), y_lim=[-200, 20])

### vel2xyz

In [ ]:
import os
import random
import numpy as np
from mintpy.utils import readfile


def vel2xyz(mask_file, vel_file, xyz_file):
    mask, atr = readfile.read(mask_file)
    mask = np.asarray(mask)

    vel, atr = readfile.read(vel_file)
    vel = np.asarray(vel)

    lon = float(atr['X_FIRST'])
    lon_step = float(atr['X_STEP'])

    lat = float(atr['Y_FIRST'])
    lat_step = float(atr['Y_STEP'])

    lon_tmp = np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1])
    lat_tmp = np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0])

    lons, lats = np.meshgrid(lon_tmp, lat_tmp)

    lons = lons[mask].reshape((-1, 1))
    lats = lats[mask].reshape((-1, 1))
    vel = vel[mask].reshape((-1, 1)) * 1000
    print('v_max : ', np.max(vel))
    print('v_min : ', np.min(vel))
    print('number of points : ', vel.shape[0])

    vels = np.concatenate([lons, lats, vel], axis=1)
    print('writing data to {}'.format(xyz_file))
    np.savetxt(xyz_file, vels, fmt='%4f')
    print('done.\n')

    
def random_downsample(xyz_file, percent):
    if os.path.isfile(xyz_file):
        data = np.loadtxt(xyz_file)
        point_num = data.shape[0]
        index = random.sample(range(point_num), int(point_num * percent))
        
        base_name = os.path.basename(xyz_file)
        dir_name = os.path.dirname(xyz_file)
        out_name = base_name.replace('.', '_ds.')
        out_file = os.path.join(dir_name, out_name)
        out_data = data[index, :]
        print('v_max : ', np.max(out_data))
        print('v_min : ', np.min(out_data))
        print('number of points : ', out_data.shape[0])
        print('writing data to {}'.format(out_file))
        np.savetxt(out_file, out_data, fmt='%4f')
        print('done.')


In [ ]:
geo_dir = '/media/ly/文件/MintPy/xsc/mintpy/geo'

mask_file = os.path.join(geo_dir, 'geo_maskTempCoh.h5')
vel_file = os.path.join(geo_dir, 'geo_velocity.h5')

out_vel_file = os.path.join(geo_dir, 'vel.txt')

vel2xyz(mask_file, vel_file, out_vel_file)
random_downsample(out_vel_file, 0.8)

### prepare data for `make_kmz_timeseries.py` and `make_kmz.py`

In [ ]:
geo_dir = '/media/ly/文件/MintPy/xsc/mintpy/geo'

mask_file = os.path.join(geo_dir, 'geo_maskTempCoh.h5')
vel_file = os.path.join(geo_dir, 'geo_velocity.h5')
ts_file = os.path.join(geo_dir, 'geo_timeseries_tropHgt_ramp_demErr.h5')

out_vel_file = os.path.join(geo_dir, 'vels.txt')
out_ts_file = os.path.join(geo_dir, 'ts.txt')

prep_data_for_kmz(mask_file, vel_file, ts_file, out_vel_file, out_ts_file)

### transection

In [ ]:
%matplotlib inline
import os
import matplotlib.pyplot as plt
from mintpy.plot_transection import transectionViewer

# visualization functions
def plot_transection(fname='./velocity.h5', start_yx=None, end_yx=None):
    cmd = 'plot_transection.py {} --noverbose --figsize 9 4 '.format(fname)
    if start_yx is not None:
        cmd += ' --start-yx {} {}'.format(start_yx[0], start_yx[1])
    if end_yx is not None:
        cmd += ' --end-yx {} {}'.format(end_yx[0], end_yx[1])
    print(cmd)
    obj = transectionViewer(cmd)
    obj.configure()
    obj.plot()
    return obj

In [ ]:
# proj_dir = '/media/ly/文件/xsc/mintpy/'
# os.chdir(proj_dir)
# obj = plot_transection('./geo/geo_velocity.h5', start_yx=[712, 2400], end_yx=[724, 2435])